<a href="https://colab.research.google.com/github/Yash-Narnaware/PG-Lab/blob/main/Yash_PG_Lab_assignment_5b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Load and preprocess the data**

In [ ]:
import gensim
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import re
import string
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK stopwords and punkt tokenizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df = pd.read_csv("/content/IMDB Dataset.csv")
print(df.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [ ]:
# Load stopwords
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize text
    tokens = word_tokenize(text.lower())

    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]

    # Join the tokens back into a single string
    return ' '.join(tokens)

# Apply preprocessing to the 'review' column
df['review'] = df['review'].apply(preprocess_text)



<ipython-input-11-f02bededea82>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [ ]:
# review_text = df.review.apply(gensim.utils.simple_preprocess)
review_text = df['review'].apply(gensim.utils.simple_preprocess)
print(review_text)

0        [one, reviewers, mentioned, watching, oz, epis...
1        [wonderful, little, production, filming, techn...
2        [thought, wonderful, way, spend, time, hot, su...
3        [basically, theres, family, little, boy, jake,...
4        [petter, matteis, love, time, money, visually,...
                               ...                        
49995    [thought, movie, right, good, job, wasnt, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [catholic, taught, parochial, elementary, scho...
49998    [im, going, disagree, previous, comment, side,...
49999    [one, expects, star, trek, movies, high, art, ...
Name: review, Length: 50000, dtype: object


In [ ]:
# import numpy as np

# # Calculate the length of each sequence (each tokenized review)
# review_lengths = [len(review) for review in review_text]

# # Compute the 90th percentile for maxlen
# maxlen = int(np.percentile(review_lengths, 90))

# print(f"90th percentile length: {maxlen}")


90th percentile length: 438


###Split the data and convert labels to numbers

In [ ]:
# Split data into training and testing
train_data, test_data = train_test_split(df, test_size=0.4, random_state=42)

# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['review'])
train_sequences = tokenizer.texts_to_sequences(train_data['review'])
test_sequences = tokenizer.texts_to_sequences(test_data['review'])

# Padding sequences
max_len = 438  # 90th percentile of max length
X_train = pad_sequences(train_sequences, maxlen=max_len, padding='post')
X_test = pad_sequences(test_sequences, maxlen=max_len, padding='post')
y_train = train_data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
y_test = test_data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

In [ ]:
print(X_train.shape)
print(y_train.shape)

(30000, 438)
(30000,)


##**Bulding LSTM based sentiment analysis model using custom trained Word2Vec model**

###Training Word2Vec model on IMDB reviews dataset

In [ ]:
import gensim
import pandas as pd

In [ ]:
# review_text = df.review.apply(gensim.utils.simple_preprocess)
# review_text = df['review'].apply(gensim.utils.simple_preprocess)
print(review_text)

0        [one, of, the, other, reviewers, has, mentione...
1        [wonderful, little, production, br, br, the, f...
2        [thought, this, was, wonderful, way, to, spend...
3        [basically, there, family, where, little, boy,...
4        [petter, mattei, love, in, the, time, of, mone...
                               ...                        
49995    [thought, this, movie, did, down, right, good,...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [am, catholic, taught, in, parochial, elementa...
49998    [going, to, have, to, disagree, with, the, pre...
49999    [no, one, expects, the, star, trek, movies, to...
Name: review, Length: 50000, dtype: object


In [ ]:
# Build the custom Word2Vec model for custom training. Uses CBOW by default for training.
custom_word2vec = gensim.models.Word2Vec(
    vector_size=100,
    window=10,
    min_count=3,
    workers=10
)

custom_word2vec.build_vocab(review_text, progress_per=1000)

In [ ]:
custom_word2vec.train(review_text, total_examples=custom_word2vec.corpus_count, epochs=custom_word2vec.epochs)

(42334325, 55882335)

In [ ]:
# custom_word2vec.wv.most_similar("superpower")
print(custom_word2vec.vector_size)
print(custom_word2vec)

100
Word2Vec<vocab=50210, vector_size=100, alpha=0.025>


###Building LSTM based classifier using customly trained Word2Vec model

In [ ]:
import pandas as pd
import numpy as np
import gensim

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

####creating embedding matrix

In [ ]:
# Load pre-trained Word2Vec model
# word2vec = Word2Vec.load("word2vec.model")

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

# Prepare embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in custom_word2vec.wv:
        embedding_matrix[i] = custom_word2vec.wv[word]

###Defining the model architecture

In [ ]:
# Build LSTM model
model1 = Sequential()
model1.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len, weights=[embedding_matrix], trainable=False))
model1.add(LSTM(128, return_sequences=True))
model1.add(LSTM(12))
model1.add(Dense(20, activation='relu'))
model1.add(Dropout(0.1))
model1.add(Dense(1, activation='sigmoid'))

# Compile the model
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model1.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)              │ ?                           │      29,497,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_18 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_19 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 29,497,800 (112.53 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 29,497,800 (112.53 MB)

In [ ]:
# Train the model
model1.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 31s 59ms/step - accuracy: 0.5231 - loss: 0.6880 - val_accuracy: 0.5904 - val_loss: 0.6738
Epoch 2/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 42s 62ms/step - accuracy: 0.5746 - loss: 0.6783 - val_accuracy: 0.5138 - val_loss: 0.6895
Epoch 3/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 38s 55ms/step - accuracy: 0.5168 - loss: 0.6905 - val_accuracy: 0.5181 - val_loss: 0.6869
Epoch 4/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 55ms/step - accuracy: 0.5755 - loss: 0.6624 - val_accuracy: 0.5785 - val_loss: 0.6595
Epoch 5/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 48s 69ms/step - accuracy: 0.5731 - loss: 0.6657 - val_accuracy: 0.5252 - val_loss: 0.6807
Epoch 6/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 34s 54ms/step - accuracy: 0.5649 - loss: 0.6666 - val_accuracy: 0.7033 - val_loss: 0.5844
Epoch 7/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 25s 53ms/step - accuracy: 0.6537 - loss: 0.6245 - val_accuracy: 0.6544 - val_loss: 0.6275
Epoch 8/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 42s 55ms/step - accuracy: 0.6620 - loss: 0.6146 - 

In [ ]:
test_model = model1.evaluate(X_test, y_test)
print("Accuracy on test dataset:",test_model[1])

625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.9003 - loss: 0.2436
Accuracy on test dataset: 0.8981000185012817


##**Bulding LSTM based sentiment analysis model using pretrained GloVe embeddings**

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.models import Sequential

###download and unzip GloVe embedding model

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2024-09-07 11:00:25--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-09-07 11:00:25--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.01MB/s    in 2m 39s  

2024-09-07 11:03:04 (5.18 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [ ]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


###load GloVe model of 100 dimentional vectors

In [ ]:
# Load the GloVe word embeddings
embedding_index = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = vector

print(f"Loaded {len(embedding_index)} word vectors.")

Loaded 400000 word vectors.


###Create embedding matrix

In [ ]:
# Define the word index
word_index = tokenizer.word_index

# Parameters
embedding_dim = 100  # Use 100-dimensional GloVe vectors
vocab_size = len(word_index) + 1  # word_index is from your tokenizer

# Create an embedding matrix with random initialization
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Fill embedding matrix with GloVe vectors
for word, idx in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[idx] = embedding_vector  # Assign GloVe vector for the word


###Defining model architecture and training the model

In [ ]:
# Build the LSTM model with pretrained GloVe embeddings
model2 = Sequential()

# Add embedding layer initialized with GloVe embeddings
model2.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_dim,
                    input_length=max_len,
                    weights=[embedding_matrix],  # Use GloVe vectors
                    trainable=False))  # Set to False to keep embeddings static

model2.add(LSTM(128, return_sequences=True))
model2.add(LSTM(12))
model2.add(Dense(20, activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(1, activation='sigmoid'))

# Compile the model
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model2.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │       9,832,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,832,600 (37.51 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 9,832,600 (37.51 MB)

In [ ]:
#training the model
history = model2.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 27s 51ms/step - accuracy: 0.4967 - loss: 0.6934 - val_accuracy: 0.5083 - val_loss: 0.6927
Epoch 2/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 43s 55ms/step - accuracy: 0.5133 - loss: 0.6920 - val_accuracy: 0.4970 - val_loss: 0.6924
Epoch 3/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 21s 45ms/step - accuracy: 0.5174 - loss: 0.6900 - val_accuracy: 0.5174 - val_loss: 0.6900
Epoch 4/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 46ms/step - accuracy: 0.5292 - loss: 0.6856 - val_accuracy: 0.7826 - val_loss: 0.5018
Epoch 5/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 34s 72ms/step - accuracy: 0.7969 - loss: 0.4678 - val_accuracy: 0.8537 - val_loss: 0.3534
Epoch 6/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 29s 45ms/step - accuracy: 0.8504 - loss: 0.3521 - val_accuracy: 0.8643 - val_loss: 0.3221
Epoch 7/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 42s 47ms/step - accuracy: 0.8747 - loss: 0.3082 - val_accuracy: 0.8723 - val_loss: 0.3065
Epoch 8/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 46ms/step - accuracy: 0.8830 - loss: 0.2822 - 

In [ ]:
test_model2 = model2.evaluate(X_test, y_test)
print("Accuracy on test dataset:",test_model2[1])

625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.8849 - loss: 0.4250
Accuracy on test dataset: 0.8838499784469604


##**Bulding LSTM based sentiment analysis model using pretrained FastText word embeddings**

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

###Download and unzip FastText embedding model

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip

--2024-09-07 11:19:36--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.14, 3.163.189.51, 3.163.189.96, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  90.0MB/s    in 11s     

2024-09-07 11:19:46 (60.9 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]



In [ ]:
!unzip wiki-news-300d-1M.vec*.zip

Archive:  wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   


###Load FastText embedding model

In [ ]:
# Initialize an empty dictionary to hold the word vectors
embedding_index = {}

# Load the FastText word embeddings file
with open('wiki-news-300d-1M.vec', encoding='utf-8') as f:
    for line in f:
        # Split each line into word and vector components
        values = line.split()
        word = values[0]  # The word
        vector = np.asarray(values[1:], dtype='float32')  # The vector representation
        embedding_index[word] = vector  # Add to the dictionary

print(f"Loaded {len(embedding_index)} word vectors.")


Loaded 999995 word vectors.


###Create embedding matrix

In [ ]:
# Define embedding dimension and vocab size
embedding_dim = 300  # Use the dimension of your FastText vectors
vocab_size = len(word_index) + 1  # Add 1 for padding

# Initialize embedding matrix with zeros
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Populate the embedding matrix with FastText vectors
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector  # Assign the FastText vector


###Defining model architecture and training the model

In [ ]:
# Define the LSTM model
model3 = Sequential()

# Add FastText embedding layer (set trainable=False to freeze FastText weights)
model3.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_dim,
                    input_length=max_len,
                    weights=[embedding_matrix],
                    trainable=False))  # Keep FastText embeddings static

# Add LSTM layers
model3.add(LSTM(128, return_sequences=True))
model3.add(LSTM(12))
# model3.add(Dense(20, activation='relu'))
# model3.add(Dropout(0.1))

# Add a dense output layer with sigmoid activation for binary classification
model3.add(Dense(1, activation='sigmoid'))

# Compile the model
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# View model summary
model3.summary()


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)              │ ?                           │      29,497,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_16 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_17 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 29,497,800 (112.53 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 29,497,800 (112.53 MB)

In [ ]:
#Training the model
history = model3.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 27s 51ms/step - accuracy: 0.5250 - loss: 0.6901 - val_accuracy: 0.5292 - val_loss: 0.6836
Epoch 2/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 45s 59ms/step - accuracy: 0.5231 - loss: 0.6878 - val_accuracy: 0.4994 - val_loss: 0.6933
Epoch 3/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 36s 49ms/step - accuracy: 0.5045 - loss: 0.6931 - val_accuracy: 0.5099 - val_loss: 0.6923
Epoch 4/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 42s 52ms/step - accuracy: 0.5078 - loss: 0.6912 - val_accuracy: 0.5332 - val_loss: 0.6757
Epoch 5/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 49s 69ms/step - accuracy: 0.5229 - loss: 0.6792 - val_accuracy: 0.5286 - val_loss: 0.6826
Epoch 6/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 34s 54ms/step - accuracy: 0.5238 - loss: 0.6818 - val_accuracy: 0.5306 - val_loss: 0.6702
Epoch 7/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 40s 52ms/step - accuracy: 0.5478 - loss: 0.6692 - val_accuracy: 0.5349 - val_loss: 0.6645
Epoch 8/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 42s 55ms/step - accuracy: 0.5333 - loss: 0.6831 - 

In [ ]:
test_loss, test_acc = model3.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')

625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.8888 - loss: 0.2706
Test Accuracy: 0.8891000151634216
